In [1]:
#!/usr/bin/env python
import numpy as np
from pathlib import Path
import sys

# ---------- Paths ----------
LCL_PATH  = str(Path().cwd())
ROOT_PATH = str(Path(LCL_PATH).parent.parent)
EXTERN_CMD_PATH = str(Path(ROOT_PATH))
GENERIC_PATH    = str(Path(ROOT_PATH) / "GeneticAlgorithmManager/my_code")
SPECIFIC_PATH   = str(Path(ROOT_PATH) / "extension")

print("""
root path:\t{}
local path:\t{}
generic_path:\t{}
project_specific_path:\t{}
extern_cmd_path:\t{}""".format(
    ROOT_PATH, LCL_PATH, GENERIC_PATH, SPECIFIC_PATH, EXTERN_CMD_PATH
))

# ---------- sys.path ----------
sys.path.append(ROOT_PATH)
sys.path.append(LCL_PATH)
sys.path.append(GENERIC_PATH)
sys.path.append(SPECIFIC_PATH)

# If you want to also use sys_remove_modules, uncomment:
# from sys_function import *

# ---------- Core imports ----------
from imshow import *
from algoritm_genetic import *
from builds.ttp_generator import TTPGenerator

# Init population
from extension.init_population.my_code.recursiv_init_population import *
from extension.init_population.my_code.init_hybrid_population_MATEI import InitPopulationHybrid

# Crossover operators
from extension.crossover.my_code.crossover import *
from extension.crossover.my_code.binary_crossover import *
from extension.crossover.my_code.ox_crossover import *
from extension.crossover.my_code.crossover_kp_MATEI import CrossoverMateiKP
from extension.crossover.my_code.crossover_tsp_MATEI import CrossoverMateiTSP

# Fitness
from extension.fitness.my_code.fitness import *
from extension.fitness.my_code.fitness_ttp import *
from extension.fitness.my_code.fitness_ttpMATEI import TTPFitness

# Metrics
from extension.metrics.my_code.metrics import *
from extension.metrics.my_code.metrics_ttp import *
from extension.metrics.my_code.metrics_ttpMATEI import MetricsTTPMATEI

# Mutation operators
from extension.mutate.my_code.mutate import *
from extension.mutate.my_code.mutate_scramble import *
from extension.mutate.my_code.binary_mutate import *
from extension.mutate.my_code.mutate_kp_MATEI import MutateMateiKP
from extension.mutate.my_code.mutate_tsp_MATEI import MutateMateiTSP

# Parent selection & stress
from extension.select_parent.my_code.select_parent import *
from extension.stres.my_code.stres_ttp_MATEI import *

# Local search & repair
from extension.local_search_algorithms.two_opt import *
from extension.local_search_algorithms.factory import LocalSearchFactory
from extension.ga_elite_local_search import GeneticAlgorithmWithEliteSearch
from extension.kp_repair.horizontal_kp_repair import HorizonKPRepair
from extension.local_search_algorithms.ttp_vnd import TTPVNDLocalSearch
from extension.kp_repair.cheap_kp_repair import FastKPRepair
from extension.kp_repair.cheap_mutate_repair import MutateKPWithRepairAndStress
from extension.kp_repair.cheap_crossover_repair import CrossoverKPWithRepair
from extension.kp_repair.multi_kp_repair import MultiStrategyKPRepair

# ---------- Dataset ----------
ttp_generator = TTPGenerator("{}/data".format(LCL_PATH))
dataset = ttp_generator(
    nodes_file="NODE_COORD_SECTION.csv",
    items_file="ITEMS_SECTION.csv"
)
dataset



root path:	c:\Users\Matei\MLProjects\UAIC-MIAO1\Sem-1\Metode-inspirate-din-natura\Homeworks\MIN
local path:	c:\Users\Matei\MLProjects\UAIC-MIAO1\Sem-1\Metode-inspirate-din-natura\Homeworks\MIN\laborator_1\TTP
generic_path:	c:\Users\Matei\MLProjects\UAIC-MIAO1\Sem-1\Metode-inspirate-din-natura\Homeworks\MIN\GeneticAlgorithmManager\my_code
project_specific_path:	c:\Users\Matei\MLProjects\UAIC-MIAO1\Sem-1\Metode-inspirate-din-natura\Homeworks\MIN\extension
extern_cmd_path:	c:\Users\Matei\MLProjects\UAIC-MIAO1\Sem-1\Metode-inspirate-din-natura\Homeworks\MIN


{'GENOME_LENGTH': 280,
 'distance': array([[ 0., 20., 24., ..., 43., 34., 18.],
        [20.,  0., 18., ..., 36., 28.,  9.],
        [24., 18.,  0., ..., 20., 11., 10.],
        ...,
        [43., 36., 20., ...,  0.,  9., 29.],
        [34., 28., 11., ...,  9.,  0., 20.],
        [18.,  9., 10., ..., 29., 20.,  0.]], shape=(280, 280)),
 'coords': array([[288, 149],
        [288, 129],
        [270, 133],
        [256, 141],
        [256, 157],
        [246, 157],
        [236, 169],
        [228, 169],
        [228, 161],
        [220, 169],
        [212, 169],
        [204, 169],
        [196, 169],
        [188, 169],
        [196, 161],
        [188, 145],
        [172, 145],
        [164, 145],
        [156, 145],
        [148, 145],
        [140, 145],
        [148, 169],
        [164, 169],
        [172, 169],
        [156, 169],
        [140, 169],
        [132, 169],
        [124, 169],
        [116, 161],
        [104, 153],
        [104, 161],
        [104, 169],
        [ 90

In [2]:
from builder_ttp import (
    build_ttp_ga,
    BASE_DATASET,
    GA_SIMPLE,
    GA_SIMPLE_VND,
    GA_FULL,
    GA_NO_MULTI_REPAIR,
    GA_NO_STRESS,
    GA_NO_TSP_LS,
    GA_NO_KP_LS,
)

# Make sure dataset has scalar TTP parameters used by builder_ttp
dataset.update(BASE_DATASET)

dataset["W"], dataset["R"], dataset["v_min"], dataset["v_max"], dataset["GENOME_LENGTH"]


(25936, 5.61, 0.1, 1.0, 280)

In [ ]:
import pandas as pd
import time
from pathlib import Path

# Ensure logs dir exists
Path(f"{LCL_PATH}/logs").mkdir(exist_ok=True)

EXPERIMENTS = [
    GA_SIMPLE,
    GA_SIMPLE_VND,
    GA_FULL,
    GA_NO_MULTI_REPAIR,
    GA_NO_STRESS,
    GA_NO_TSP_LS,
    GA_NO_KP_LS,
    GA_MULTI_GENTLE,
]

results = []

for cfg in EXPERIMENTS:
    cfg_id  = cfg["id"]
    desc    = cfg.get("description", "")
    print("\n" + "=" * 80)
    print(f"Running config: {cfg_id}")
    print(desc)
    print("=" * 80)

    log_file      = f"{LCL_PATH}/logs/{cfg_id}_history.csv"
    extern_cmd    = f"{EXTERN_CMD_PATH}/extern_command.cmd"

    # Build GA with this configuration
    ga = build_ttp_ga(dataset, cfg, extern_cmd, log_file)

    # Run GA
    t0 = time.time()
    best_individ = ga()          # this calls GeneticAlgorithm.__call__()
    t1 = time.time()

    elapsed = t1 - t0

    # Evaluate best individual with the same Metrics object you used in training
    metric_values = ga.metric(best_individ)
    # metric_values keys: "profit", "time", "distance", "weights", "score"

    row = {
        "id": cfg_id,
        "description": desc,
        "score": float(metric_values["score"]),
        "profit": float(metric_values["profit"]),
        "time_travel": float(metric_values["time"]),
        "distance": float(metric_values["distance"]),
        "weight": float(metric_values["weights"]),
        "runtime_sec": elapsed,

        "population_size": cfg["population_size"],
        "generations": cfg["generations"],
        "mutation_rate": cfg["mutation_rate"],
        "crossover_rate": cfg["crossover_rate"],
        "use_multi_repair": cfg["use_multi_repair"],
        "use_stress": cfg["use_stress"],
        "use_vnd": cfg["use_vnd"],
        "use_tsp_ls": cfg["use_tsp_ls"],
        "use_kp_ls": cfg["use_kp_ls"],
    }
    results.append(row)

# Put everything in a nice table
results_df = pd.DataFrame(results)

# Sort by score (or profit) depending on what you treat as objective
results_df_sorted = results_df.sort_values("score", ascending=False)

display(results_df_sorted)

# Save summary to CSV for later plotting / paper tables
summary_path = f"{LCL_PATH}/logs/summary_variants.csv"
results_df_sorted.to_csv(summary_path, index=False)
print(f"\nSaved summary to: {summary_path}")



Running config: ga_simple
Simple GA: cheap repair only, no stress, no VND LS
setParameters: {'GENOME_LENGTH': 280, 'GENERATIONS': 50, 'POPULATION_SIZE': 100, 'MUTATION_RATE': 0.08, 'CROSSOVER_RATE': 0.7, 'SELECT_RATE': 0.5, 'W': 25936, 'v_min': 0.1, 'v_max': 1.0, 'R': 5.61, 'ELITE_SIZE': 40, 'ELITE_FREQ': 8}
Hybrid mixed TTP population = (100, 2, (280, 280))
Name:ga_simple, Generatia: 0, score: 3501.326, profit: 34263.0, time: 5483.364, distance: 3808.0, weight: 25863.0, best_fitness: 579848.714, 
Name:ga_simple, Generatia: 1, score: 3799.664, profit: 33628.0, time: 5316.994, distance: 3431.0, weight: 25928.0, best_fitness: 316190.273, 
Name:ga_simple, Generatia: 2, score: 5666.121, profit: 33628.0, time: 4984.292, distance: 2771.0, weight: 25928.0, best_fitness: 208904.192, 
Name:ga_simple, Generatia: 3, score: 7570.411, profit: 31732.0, time: 4306.879, distance: 2839.0, weight: 24132.0, best_fitness: 173587.861, 
Name:ga_simple, Generatia: 4, score: 8121.188, profit: 29434.0, time: 

AttributeError: 'GeneticAlgorithmWithEliteSearch' object has no attribute 'metric'